#### Setting up
We want to be able to view the tables available and the attributes of the tables. (Your tutor had investigated using freewha.com and the inbuilt MyPHPAdmin but the downside is you can't view the attributes when writing your queries)

This code loads a sql extension for jupyter. Also hides the connected database and other annoying error messages.

We are going to download the Chinook database from github using a command wget, commands which are executed via bash have a ! in front of them.
We are then going to load this database and connect to it.

Adding a sidebar to display useful database stuff.

For future reference if you have a link to another sqlite database you must **Change the url of DB_URL and name of DB_NAME.**

In [1]:
%reload_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

import sys
sys.path.insert(0, './.hidden')
from ipylab import JupyterFrontEnd, Panel
from ipywidgets import Tab, Text, Output, HTML, HBox, VBox, Button, Layout,ButtonStyle
app = JupyterFrontEnd()

DB_URL = "https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite?raw=true"
DB_NAME = "Chinook"
DB_FILE = "Chinook.db"

!wget -q -O $DB_FILE $DB_URL
%sql sqlite:///$DB_FILE

#GET TABLE INFO

tablesList = !sqlite3 $DB_FILE .table
!sqlite3 $DB_FILE .table > .hidden/tables.txt
tables = [i for line in tablesList for i in line.split()]
tables.sort()
AllTables = " ".join([table for table in tables])

#RIGHT PANEL INFO

tblInfo = !./.hidden/Table_View.sh $DB_FILE $AllTables
tblAttr = !./.hidden/getAttributes.sh $DB_FILE $AllTables
ddl = !sqlite3 $DB_FILE .schema

from right_panel import display_right_panel
display_right_panel(app, tblInfo, tblAttr, ddl)

#ADD ER DIAGRAM
IMG_FILE = f"erd_{DB_NAME}.png"
!eralchemy -i sqlite:///$DB_FILE -o $IMG_FILE

from image_load import load_image
load_image(app, IMG_FILE)

Some Wizardry to Display Table names on the left

To execute sql into this jupyter notebook, we must either make use of the `%sql` or `%%sql` magic command. `%sql` is for a single line and `%%sql` is for an entire cell.

In [4]:
%%sql
SELECT * 
FROM Album
LIMIT 10

AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
6,Jagged Little Pill,4
7,Facelift,5
8,Warner 25 Anos,6
9,Plays Metallica By Four Cellos,7
10,Audioslave,8


Alternatively you can access the database using sqlite3 for a sqlite database and execute commands. For this to work; we get a instance of bash working + the commands to go into the sqlite executable. However this is not interactive unlike bash. The developer of the ipython-sql plugin did some work to make it output more prettily

In [10]:
%%bash
sqlite3 Chinook.db
.headers on
.mode markdown
SELECT * FROM Album
LIMIT 10;
.quit

| AlbumId |                 Title                 | ArtistId |
|---------|---------------------------------------|----------|
| 1       | For Those About To Rock We Salute You | 1        |
| 2       | Balls to the Wall                     | 2        |
| 3       | Restless and Wild                     | 2        |
| 4       | Let There Be Rock                     | 1        |
| 5       | Big Ones                              | 3        |
| 6       | Jagged Little Pill                    | 4        |
| 7       | Facelift                              | 5        |
| 8       | Warner 25 Anos                        | 6        |
| 9       | Plays Metallica By Four Cellos        | 7        |
| 10      | Audioslave                            | 8        |
